In [5]:
import os
import sys
import re

from pyspark.sql import functions as F
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StringType

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

##### Создаем датафрейм со словами

In [6]:

spark = SparkSession.builder.master('local[*]').appName('Pys').getOrCreate()
f = open(r'C:\Users\Roman Andreevich\PycharmProjects\bigD\wiki.txt','r',encoding="utf-8")
file = f.read()
file = file.replace('(','')
file = file.replace(')','')
file = re.sub('\d','',file)
fi = re.split('[ %\/,_=&?!-]',file)
df = spark.createDataFrame(fi,StringType())

1. Напишите программу, которая находит самое длинное слово.

In [7]:
df_temp = df.withColumn('len',F.length(df.value))
df_temp.sort(col('len').desc()).head()

Row(value='Rindfleischetikettierungsüberwachungsaufgabenübertragungsgesetz»"', len=65)

2. Напишите программу, которая находит среднюю длину слов.

In [8]:
df_temp.select(F.mean(col('len'))).show()

+-----------------+
|         avg(len)|
+-----------------+
|5.656381829659606|
+-----------------+



3. Напишите программу, которая находит самое частоупотребляемое слово, состоящее из латинских букв.

In [14]:
df_temp = df.filter(col('value').rlike('^[A-Za-z]'))
df_temp =df_temp.groupby('value').count()
df_temp.sort(col('count').desc()).head()

Row(value='wiki', count=12614)

4. Все слова, которые более чем в половине случаев начинаются с большой буквы и встречаются больше 10 раз.

In [10]:
# Датафрейм с общим количеством слов
df_temp=df.filter(col('value').rlike('^[А-Яа-я]*$'))
df_temp=df_temp.select(F.lower('value').alias('low'))
df_temp =df_temp.groupby(col('low')).count()
df_temp = df_temp.filter(col('count')>=10)
df_temp=df_temp.withColumnRenamed('count','count1')

# Датафрейм с раздельным количеством по регистру
df_temp_all = df.groupby(col('value')).count()

# Объединяем по словам innerjoin
df_temp = df_temp.join(df_temp_all,df_temp.low==df_temp_all.value,'inner')
df_temp =df_temp.drop('value')

# Считаем отношение больших к маленьким
df_temp = df_temp.withColumn('coef', (df_temp['count1']-df_temp['count'])/df_temp['count1'])
df_temp.filter(col('coef')>0.5).show(5)

+--------------+------+-----+------------------+
|           low|count1|count|              coef|
+--------------+------+-----+------------------+
|адмиралтейства|    27|    6|0.7777777777777778|
|    азиатского|    36|   16|0.5555555555555556|
|      азовский|    11|    2|0.8181818181818182|
|      азовской|    19|    1|0.9473684210526315|
|     алексеева|    34|    1|0.9705882352941176|
+--------------+------+-----+------------------+
only showing top 5 rows



6. Напишите программу, которая с помощью статистики определяет устойчивые сокращения вида т.п., н.э., ...

In [11]:
df.filter(col('value').rlike('^[а-я]\.[а-я]\.$')).show(5)

+-----+
|value|
+-----+
| н.э.|
| т.д.|
| м.н.|
| с.ш.|
| в.д.|
+-----+
only showing top 5 rows



7. Напишите программу, которая с помощью статистики находит имена собственные, употребляющиеся в статьях.

In [12]:
df.filter(col('value').rlike('^[А-Я][а-я]*')).show(5)

+-----------+
|      value|
+-----------+
|     Литва́|
| Лито́вская|
|Респу́блика|
|   Северной|
|     Европе|
+-----------+
only showing top 5 rows

